<a href="https://colab.research.google.com/github/ram130849/Deep_Learning_Systems_Assignments/blob/main/TensorFlow/Ramki/E533_DLS_Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install IPython
!pip install librosa

In [ ]:
import numpy as np
from IPython.display import Audio
import librosa
from google.colab import drive
import tensorflow as tf
from tensorflow.keras import backend as K
from librosa.core import stft,istft
from tensorflow.keras.layers import GRU,Dropout
from tensorflow.keras.layers import Conv1D,Conv2D,MaxPooling2D,Dropout,Dense,Flatten,BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import os
import soundfile as sf
drive.mount('/content/drive/')

Q1: Data Augmentation

In [ ]:
cifar10 = tf.keras.datasets.cifar10

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
 X_train, X_val, Y_train, Y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=1)
 print(X_train.shape)
 print(X_val.shape)
 print(Y_train.shape)
 print(Y_val.shape)

In [ ]:
def normalize_tensor(tensor): 
    tensor = tf.math.divide(
          tf.subtract(
              tensor, 
              tf.reduce_min(tensor)
          ), 
          tf.subtract(
              tf.reduce_max(tensor), 
              tf.reduce_min(tensor)
          )
        )
    return 2*tensor-1

In [ ]:
X_train_n2, X_val_n2 = X_train/255.0, X_val/ 255.0

In [ ]:
Y_train, Y_val = Y_train.flatten(), Y_val.flatten()
layer = tf.keras.layers.Normalization(mean=0, variance=1)
X_train_n1 = normalize_tensor(X_train)
X_val_n1 = normalize_tensor(X_val)

In [ ]:
he_initializer = tf.keras.initializers.HeUniform(seed=None)

In [ ]:
model = tf.keras.Sequential()
model.add(Conv2D(10, (5, 5), strides=(1,1), activation='relu', input_shape=(32, 32, 3),kernel_initializer=he_initializer))
model.add(MaxPooling2D((2, 2),strides=(2,2)))
model.add(Conv2D(10, (5, 5), activation='relu',kernel_initializer=he_initializer))
model.add(MaxPooling2D((2, 2),strides=2))
model.add(Flatten())
model.add(Dense(units=20,activation='relu',kernel_initializer=he_initializer))
model.add(Dense(units=10,activation='softmax',kernel_initializer=he_initializer))

In [ ]:
adam = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=adam,loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
r = model.fit(X_train_n1, Y_train, validation_data=(X_val_n1, Y_val), batch_size=256, epochs=150)

In [ ]:
aug_x_train1 = np.minimum(1.1*X_train_n2, 1)
aug_x_train1 = normalize_tensor(aug_x_train1)
aug_x_train2 = 0.9*X_train_n2
aug_x_train2 = normalize_tensor(aug_x_train2)
aug_x_train3 = tf.image.flip_left_right(X_train_n2)
aug_x_train3 = normalize_tensor(aug_x_train3)
aug_x_train4 = X_train_n2
aug_x_train4 = normalize_tensor(aug_x_train4)

In [ ]:
X_train_aug = tf.concat([aug_x_train1,aug_x_train2,aug_x_train3,aug_x_train4],0)

In [ ]:
X_train_aug.shape

In [ ]:
Y_train_aug = tf.concat([Y_train,Y_train,Y_train,Y_train],0)

In [ ]:
X_train_n1 = X_train_aug
X_val_n1 = normalize_tensor(X_val)

In [ ]:
model1 = tf.keras.Sequential()
model1.add(Conv2D(10, (5, 5), strides=(1,1), activation='relu', input_shape=(32, 32, 3),kernel_initializer=he_initializer))
model1.add(MaxPooling2D((2, 2),strides=(2,2)))
model1.add(Conv2D(10, (5, 5), activation='relu',kernel_initializer=he_initializer))
model1.add(MaxPooling2D((2, 2),strides=2))
model1.add(Flatten())
model1.add(Dense(units=20,activation='relu',kernel_initializer=he_initializer))
model1.add(Dense(units=10,activation='softmax',kernel_initializer=he_initializer))

In [ ]:
adam = tf.keras.optimizers.Adam(learning_rate=0.001)
model1.compile(optimizer=adam,loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
r1 = model1.fit(X_train_n1, Y_train_aug, validation_data=(X_val_n1, Y_val), batch_size=512, epochs=150)

In [ ]:
val_acc1 = r.history['val_accuracy']
val_acc2 = r1.history['val_accuracy']

In [ ]:
epochs = range(1, 151)

plt.figure()
plt.plot(epochs, val_acc1, 'bo', label='Validation accuracy for Normal Data')
plt.plot(epochs, val_acc2, 'b', label='Validation accuracy for Augmented Data')
plt.title('Validation accuracy for both Normal and Augmented Dataset')
plt.legend()

plt.show()

In [ ]:
# !unzip '/content/drive/MyDrive/DLS_DATA/homework3.zip' -d '/content/drive/MyDrive/DLS_DATA/timit-hw3'

In [ ]:
path = '/content/drive/MyDrive/DLS_DATA/timit-hw3/timit-homework/tr'

X = []
S = []
N = []

for f in os.listdir(path):
    if(os.path.isfile(os.path.join(path,f))):
        if(f[:3]=='trn'):
          s, sr = librosa.load(os.path.join(path,f),sr=None)
          trn_stft=librosa.stft(s, n_fft=1024, hop_length=512)
          N.append(abs(trn_stft))
        elif(f[:3]=='trs'):
          s, sr = librosa.load(os.path.join(path,f),sr=None)
          trs_stft=librosa.stft(s, n_fft=1024, hop_length=512)
          S.append(abs(trs_stft))
        else:
          s, sr = librosa.load(os.path.join(path,f),sr=None)
          trx_stft=librosa.stft(s, n_fft=1024, hop_length=512)
          X.append(abs(trx_stft))

In [ ]:
print(N[-1].shape)
print(S[-1].shape)
print(X[-1].shape)

In [ ]:
max_len = X[0].shape[1]
for k in range(1,len(X)):
    if(X[k].shape[1]>max_len):
        max_len = X[k].shape[1]
print(max_len)

In [ ]:
M_arr = []
for i in range(len(N)):
    # print("Before:",S[i].shape,N[i].shape)
    if(S[i].shape[1]<N[i].shape[1]):
        S[i] = np.pad(S[i],((0,0),(0,N[i].shape[1]-S[i].shape[1])),mode='constant')
    elif(S[i].shape[1]>N[i].shape[1]):
        N[i] = np.pad(N[i],((0,0),(0,S[i].shape[1]-N[i].shape[1])),mode='constant')
    # print("After:",S[i].shape,N[i].shape)
    M = np.where((S[i]>N[i]),1,0)
    M_arr.append(M)

In [ ]:
for i in range(len(X)):
    X[i] = np.pad(X[i],((0,0),(0,max_len-X[i].shape[1])),mode='constant')
    X[i] = X[i].T
    M_arr[i] = np.pad(M_arr[i],((0,0),(0,max_len-M_arr[i].shape[1])),mode='constant')
    M_arr[i] = M_arr[i].T

In [ ]:
print(M_arr[0].shape)
print(M_arr[1].shape)
print(M_arr[2].shape)
print(M_arr[3].shape)

In [ ]:
X = np.dstack(X)
X = np.rollaxis(X,-1)
M_arr = np.dstack(M_arr)
M_arr = np.rollaxis(M_arr,-1)
print(X.shape)
print(M_arr.shape)

In [ ]:
X = tf.convert_to_tensor(X, dtype=tf.float32)
M_arr = tf.convert_to_tensor(M_arr, dtype=tf.float32)
print(X.shape)
print(M_arr.shape)

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(max_len,513),batch_size=10))
model.add(tf.keras.layers.GRU(513, return_sequences=True, input_shape =(max_len,513), dropout=0.3))
model.add(tf.keras.layers.GRU(513, return_sequences=True, dropout=0.3))
model.add(Dropout(0.3))
model.add(Dense(units=1024,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(units=513,activation='relu'))
model.compile(optimizer='adam',loss = 'mean_squared_error')

In [ ]:
model.summary()

In [ ]:
model.fit(X,M_arr,batch_size=10,epochs=100)